# Live coding with pygame and ipython

Since I have seen Notch livecode a game where he could change the code without restarting the binary, I was jealous of this kind of live programming. Actually, I was so envious that at the time I had made proofs of concept in C (not cross-platform, unforntunately) but never did anything out of it. I somehow suspected it should be much easier to do in Python.

I recently had to reinstall IPython notebook. When I first tried it it was a bit buggy but I really wanted to do all my devs in this form. A scientist friend told me that it should probably be possible, but that it would be a bad idea for games. Hey, "workable but bad idea" sounds like half of my weekend projects!

Let's see how to get there.

In [5]:
import pygame

pygame.init()
size = (500,500)
disp=pygame.display.set_mode(size, pygame.HWSURFACE | pygame.DOUBLEBUF)
running = True
x=10
y=20
dirx=1
color = (255,255,255)

while( running ):
    disp.fill((0, 0, 0))
    pygame.draw.circle(disp, color, (x, y), 15)
    pygame.display.flip()
    for event in pygame.event.get():
        if event.type == pygame.QUIT or event.type == pygame.KEYDOWN:
            running = False
    if x>500:
        dirx=-1
    if x<0:
        dirx=1
    x+=dirx
    pygame.time.delay(5)
pygame.quit()

This was my first test. Just a circle bouncing left and right, an event loop that exits on any key press. My goal was to change the color of the ball without restarting the program, so I tried:

In [6]:
color = (255,0,0)

But as you see, it becomes tagged as "running" but actually waits for the previous cell to finish executing to start running its code, and then exit. This is not what we want. 

The next step was to run the mainloop inside a thread instead:

In [7]:
import threading

pygame.init()
size = (500,500)
disp=pygame.display.set_mode(size, pygame.HWSURFACE | pygame.DOUBLEBUF)

running = True
x=10
y=20
dirx=1
color = (255,0,0)

def mainloop():
    global running
    global x,y,dirx,color
    while( running ):
        disp.fill((0, 0, 0))
        pygame.draw.circle(disp, color, (x, y), 15)
        pygame.display.flip()
        for event in pygame.event.get():
            if event.type == pygame.QUIT or event.type == pygame.KEYDOWN:
                running = False
        if x>500:
            dirx=-1
        if x<0:
            dirx=1
        x+=dirx
    pygame.quit()

th = threading.Thread(target = mainloop)
th.start()

I recommend resizing the notebook window to half the size of the screen and put the pygame display on the other half. And then, execute the next cell:

In [8]:
color = (255,0,0)

In [9]:
color = (0,255,0)

Bingo! It changed color in the same window! Now you probably have heard, or you may have the (sane) feeling that changing variables from one thread to the other is not the cleanest thing to do. And you are right, but python is famous for having a big problem when it comes to threading: the GIL (Global  Interpreter Lock) that actually prevents most operations to operate concurrently. 
 
This is a problem if you want to use parallelism to improve the performances of your program but in our case it allows us to read and write to the same objects accross threads relatively safely.

## Let's make it more generic
This is nice but for now it is still impossible to change the objects displayed or their behavior without restarting the program. Let's see if we can go further by creating a more classical game architecture: we will make the game logic very light and generic and create Ball objects that will be responsible for their own rendering and updating.

In [10]:
import pygame
import threading

class Game:
    def __init__(self):
        self.size = (500,500)
        self.running = True
        self.scene = list()
        
    def render(self):
        self.disp.fill((0,0,0))
        for obj in self.scene:
            obj.render(self.disp)
        pygame.display.flip()

        
    def mainloop(self):
        pygame.init()
        self.disp=pygame.display.set_mode(self.size, pygame.HWSURFACE | pygame.DOUBLEBUF)
        while( self.running ):
            self.render()
            for event in pygame.event.get():
                if event.type == pygame.QUIT or event.type == pygame.KEYDOWN:
                    self.running = False
            for obj in self.scene:
                obj.update()
        pygame.quit()



And let's simplify the game class to become very generic:

In [11]:
class Ball:
    def __init__(self, x, y, color):
        self.x = x
        self.y = y
        self.color = color
        self.dirx=1
        
    def render(self, display):
        pygame.draw.circle(display, self.color, (self.x, self.y), 20)
        
    def update(self):
        if self.x>500:
            self.dirx=-1
        if self.x<0:
            self.dirx=1
        self.x+=self.dirx

In [12]:
game = Game()
th = threading.Thread(target = game.mainloop)
th.start()

It looks like it is working. It shows an empty window, with no object, which is normal as we have added none. Let's try to add one:

In [13]:
game.scene.append(Ball(15,30,(255,255,255)))

Victory! We can still change the characteristics of that object:

In [14]:
game.scene[0].color = (255,0,0)

Or add some more:

In [15]:
game.scene.append(Ball(20,80,(0,0,255)))

Now, can we define a different kind of object and add it without restarting the program?

In [16]:
class Ball2(Ball):
    def __init__(self, x, y, color):
        super().__init__( x, y, color)
        
    def update(self):
        super().update()
        self.y+=self.dirx

In [17]:
game.scene.append(Ball2(20,150,(255,255,0)))

You bet we can!

How about redefining the behavior of an already running object? Well it is possible, because in python, methods are actually considered like member variables that can be changed on the fly. One can change the method of a Class directly (and change the content of the method of the instances that will be instantiated in the future) and even change classes of instances, bringing the weird situation where some instances have different content in their methods.

It is not really recommended but this is how to do it:

In [18]:
class Ball(Ball):
    def update(self):
        if self.x>500:
            self.dirx=-1
        if self.x<0:
            self.dirx=1
        self.x+=self.dirx
        self.y+=self.dirx
        
        
import types

game.scene[0].update = types.MethodType(Ball.update, game.scene[0])